In [1]:
#from fuzzywuzzy import process

import matplotlib.pyplot as plt

import requests
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import fuzz, process
import Levenshtein
import os
import glob

import pandas_profiling
from pandas_profiling.utils.cache import cache_file

In [2]:
def read_all_excel_cwd():
    
    # Might need to try and str.lower() columns before indexing by date
    # extracted_cols = list(df1['Behaviors'].keys())
    # also need to deal with getting behaviors sheets
#    df = pd.concat(pd.read_excel('2018_Sales_Total.xlsx', sheet_name=None), ignore_index=True)
    extracted_df = pd.DataFrame()
    test = pd.DataFrame()
    x=0
    test = pd.concat([extracted_df, test])

    path = os.getcwd()
    excel_files = glob.glob(os.path.join(path, "*.xlsx"))
    print(excel_files)
    
    for file in excel_files:
        print(len(excel_files))
        x += 1
        print('finished ', x, ' files')
        df = pd.concat(pd.read_excel(file, sheet_name=None), ignore_index=True)
        print(df.keys())
        extracted_df = extracted_df.append(df)

#    extracted_df = df
    extracted_df.columns = map(str.lower, extracted_df.columns)
    extracted_cols = extracted_df.columns #still need to return this later
    
    print('should be df next')
#    print(extracted_df)
#    print(extracted_df)        
    return extracted_df, extracted_cols

#        df.columns = map(str.lower, df.columns)  
#        print(df.columns)


In [3]:
extracted_df, extracted_cols = read_all_excel_cwd()

['/home/eyesforcomputers/Scripts/git/DataScienceCapstone2/WQ_Graphs.xlsx', '/home/eyesforcomputers/Scripts/git/DataScienceCapstone2/FHA_BF90_RM_A_05312021.xlsx', '/home/eyesforcomputers/Scripts/git/DataScienceCapstone2/GO.xlsx']
3
finished  1  files
Index(['Date', 'Elopement', 'Eloping', 'Self-Injurous Behavior', 'Eloped',
       'Dropping', 'Agg ', 'Aggression', 'Non-Compliance', 'Mouthing',
       'Protest', 'Non-comp', 'Agg', 'Crying',
       'Off task during 1:1 with teacher', 'Accidents AM', 'Accidents PM',
       'Phase Change', 'Unnamed: 4', 'Mands', 'Unnamed: 1', 'Target'],
      dtype='object')
3
finished  2  files
Index(['HUD PROJECT NUMBER', 'PROPERTY NAME', 'PROPERTY STREET',
       'PROPERTY CITY', 'PROPERTY STATE', 'PROPERTY ZIP', 'UNITS',
       'Unnamed: 7', 'Unnamed: 8', 'ORIGINAL MORTGAGE AMOUNT', 'Unnamed: 10',
       'Unnamed: 11', 'TERM IN MONTHS', 'INTEREST RATE',
       'CURRENT PRINCIPAL AND INTEREST', 'AMORITIZED PRINCIPAL BALANCE',
       'HOLDER NAME', 'HOLDE

In [4]:
bx_list = ['date','aggression', 'elope', 'non-compliance', 'sib', 'protesting', 'agg'  ] #'agg','repeated' #'date'

In [5]:
# need to add extracted_col in 
def fuzz_match():

    matches = []
    match_df = pd.DataFrame()
    cols=['bx', 'original_col_name', 'fuzz_pr']

    
    for bx in bx_list:
        print('working on: ', bx)
    
        for item in extracted_cols:
           
            fuzz_pr = fuzz.partial_ratio(item, bx)
        
            df_res = pd.DataFrame(data={'bx': bx, 'original_col_name' : item, 'fuzz_pr':fuzz_pr}, index=[i for i in range(len(extracted_df))] )
            match_df = match_df.append(df_res)

    return match_df

match_df = fuzz_match()
print('done')



working on:  date
working on:  aggression
working on:  elope
working on:  non-compliance
working on:  sib
working on:  protesting
working on:  agg
done


In [6]:
# Visually inspecting samples of the match_df by bx and fuzz_pr to determine a better match threshold
#print('starting')
# thresh = 80
#date_matches_styled = match_df.loc[(match_df.bx.str.contains('date')) & (match_df.fuzz_pr > 50) & (match_df.fuzz_pr <= 100) ].groupby('fuzz_pr', sort=True).sample(10).style.background_gradient(subset='fuzz_pr',cmap='summer_r')

#thresh 80
#agg_matches_styled = match_df.loc[(match_df.bx.str.contains('aggression')) & (match_df.fuzz_pr > 50) & (match_df.fuzz_pr <= 100) ].groupby('fuzz_pr', sort=True).sample(10).style.background_gradient(subset='fuzz_pr',cmap='summer_r')

#thresh 65
#elope_matches_styled = match_df.loc[(match_df.bx.str.contains('elope')) & (match_df.fuzz_pr > 50) & (match_df.fuzz_pr <= 100) ].groupby('fuzz_pr', sort=True).sample(10).style.background_gradient(subset='fuzz_pr',cmap='summer_r')

#thresh 90
#noncomp_matches_styled = match_df.loc[(match_df.bx.str.contains('nc')) & (match_df.fuzz_pr > 50) & (match_df.fuzz_pr <= 100) ].groupby('fuzz_pr', sort=True).sample(10).style.background_gradient(subset='fuzz_pr',cmap='summer_r')

# thresh=70
#sib_matches_styled = match_df.loc[(match_df.bx.str.contains('sib')) & (match_df.fuzz_pr > 50) & (match_df.fuzz_pr <= 100) ].groupby('fuzz_pr', sort=True).sample(10).style.background_gradient(subset='fuzz_pr',cmap='summer_r')

#print('done')

#put the thresholds in a dictionary to access later
fuzz_thresh = { 'agg_matches' : 70, 'elope_matches': 65, 'noncomp_matches' : 90} #'date_matches' : 80,


#profile_report = match_df.profile_report(explorative=True, html={'style': {'full_width': True}})
#profile_report



agg_matches = match_df.loc[(match_df.bx.str.contains('aggression')) & (match_df.fuzz_pr > 80)].drop_duplicates(subset ="original_col_name", keep ='first', inplace = False)

elope_matches = match_df.loc[(match_df.bx.str.contains('elope')) & (match_df.fuzz_pr > 65)].drop_duplicates(subset ="original_col_name", keep ='first', inplace = False)

noncomp_matches = match_df.loc[(match_df.bx.str.contains('non-compliance')) & (match_df.fuzz_pr > 90)].drop_duplicates(subset ="original_col_name", keep ='first', inplace = False)

date_matches = match_df.loc[(match_df.bx.str.contains('date')) & (match_df.fuzz_pr > 80)].drop_duplicates(subset ="original_col_name", keep ='first', inplace = False)

all_matches = pd.concat([agg_matches,elope_matches,noncomp_matches,date_matches])

print('done')

#sib_matches = match_df.loc[(match_df.bx.str.contains('sib')) & (match_df.fuzz_pr > 70)].drop_duplicates(subset ="original_col_name", keep ='first', inplace = False)

done


In [7]:
all_matches

bx original_col_name  fuzz_pr
0      aggression        aggression      100
0      aggression               agg      100
0           elope         elopement      100
0           elope           eloping       80
0           elope            eloped      100
0  non-compliance    non-compliance      100
0  non-compliance          non-comp      100
0            date              date      100
0            date            date.1      100

In [8]:
agg_matches_styled = match_df.loc[(match_df.bx.str.contains('aggression')) & (match_df.fuzz_pr > 50) & (match_df.fuzz_pr <= 100) ].groupby('fuzz_pr', sort=True).sample(10).style.background_gradient(subset='fuzz_pr',cmap='summer_r')
#agg_matches_styled

In [9]:
master_df = pd.DataFrame()

In [10]:
list(all_matches.bx)[1]

'aggression'

In [11]:
#original_col_data = extracted_df.columns
extracted_df['eloping'].notnull().sum()

34

In [16]:
# def add_to_master_df(all_matches):

def add_to_master_df(all_matches):
        
        
    for i, v in zip(all_matches.bx,all_matches.original_col_name):
        print(i,' - ', v)
        
        original_col_data = extracted_df[v]
        
        proper_col_name = i
            
        master_df[proper_col_name] = master_df.append(original_col_data) #master_df[proper_col_name].append(original_col_data)
                
        
#        print(i, v)
        
        

add_to_master_df(all_matches)



#WINNER
#    master_df = extracted_df[all_matches['original_col_name'].tolist()]
#    master_df.columns = all_matches.bx.tolist()
#    master_df.head()



#    for item in matches:
        
#    filter_bx = list(extracted_df.columns.str.contains(item[0]))
#    proper_col_name = extracted_df.columns[filter_bx][0] 
#        print(proper_col_name)#


#        original_col_data = extracted_df[item[0]]
#        proper_col_name = item[2]
#        print(proper_col_name)
#        master_df[proper_col_name] = original_col_data 
           

            
#    return master_df


aggression  -  aggression


ValueError: cannot reindex from a duplicate axis

In [65]:
master_df.set_index('date').sort_values('date')

aggression  elope  non-compliance
date                                         
2021-04-01         NaN    NaN             NaN
2021-04-02         NaN    NaN             NaN
2021-04-05         NaN    NaN             NaN
2021-04-06         NaN    NaN             NaN
2021-04-07         NaN    NaN             NaN
...                ...    ...             ...
NaT                NaN    NaN             NaN
NaT                NaN    NaN             NaN
NaT                NaN    NaN             NaN
NaT                NaN    NaN             NaN
NaT                NaN    NaN             NaN

[18475 rows x 3 columns]

In [57]:
#master_df.elope.dropna(inplace=False)

In [58]:
master_df.set_index('date').sum()

aggression        295.0
elope             240.0
non-compliance     61.0
dtype: float64

https://towardsdatascience.com/string-matching-with-fuzzywuzzy-e982c61f8a84
Need function to move through the bx_list and use process.extract on each.

hmm that means that each iteration though the list it could find the same options